In [1]:
import os
import json
import copy
import folium # version 0.5.0
import pandas as pd # version 0.20.3
import numpy as np
import seaborn as sns # version 0.8
from IPython.display import Image
import imgkit # version 0.1.8

MIDDLE_EU = [53.5775, 23.106111]
MIDDLE_CH = [46.801111, 8.226667]

WORKING_DIR = os.getcwd()
TOPOJSON_EU_PATH = os.path.join(WORKING_DIR, 'topojson', 'europe.topojson.json')
TOPOJSON_CH_PATH = os.path.join(WORKING_DIR, 'topojson', 'ch-cantons.topojson.json')
TABLES_FOLDER = os.path.join(WORKING_DIR, 'tables')
FIGURES_FOLDER = os.path.join(WORKING_DIR, 'figures')
UNEMPLOYMENT_EU_PATH = os.path.join(TABLES_FOLDER, 'tsdec450.tsv') 


VALID_EU = {'Belgium': 'BE', 'Greece': 'EL', 'Lithuania': 'LT', 
            'Portugal': 'PT', 'Bulgaria': 'BG', 'Spain': 'ES', 
            'Luxembourg': 'LU','Romania': 'RO', 'Czech Republic': 'CZ',
            'France': 'FR', 'Hungary': 'HU', 'Slovenia': 'SI',
            'Denmark': 'DK', 'Croatia': 'HR', 'Malta': 'MT', 
            'Slovakia':'SK', 'Germany': 'DE', 'Italy': 'IT', 
            'Netherlands': 'NL', 'Finland': 'FI', 'Estonia': 'EE', 
            'Cyprus': 'CY', 'Austria': 'AT', 'Sweden': 'SE',
            'Ireland': 'IE', 'Latvia': 'LV', 'Poland': 'PL', 
            'United Kingdom': 'UK'}

folium.TopoJson;

## 1
Go to the [eurostat](http://ec.europa.eu/eurostat/data/database) website and try to find a dataset that includes the european unemployment rates at a recent date.

   Use this data to build a [Choropleth map](https://en.wikipedia.org/wiki/Choropleth_map) which shows the unemployment rate in Europe at a country level. Think about [the colors you use](https://carto.com/academy/courses/intermediate-design/choose-colors-1/), how you decided to [split the intervals into data classes](http://gisgeography.com/choropleth-maps-data-classification/) or which interactions you could add in order to make the visualization intuitive and expressive. Compare Switzerland's unemployment rate to that of the rest of Europe.

The historic unemployment rate of EU and the definition of unemployment according `Eurostat` can be find  [here](http://ec.europa.eu/eurostat/web/products-datasets/-/tsdec450). For more details about coherence, accurary, meaning, among others, the metadata of the table can be find [here](http://ec.europa.eu/eurostat/cache/metadata/en/tsdec450_esmsip.htm).

The unemployment rate for Switzerland in 2016 was extracted from [here](https://www.statista.com/statistics/263707/unemployment-rate-in-switzerland/)

In [2]:
def treat_raw_unemployment(df):
    '''Filter valid years, filter valid countries, 
       replace missing by nans, split columns, filter unit'''
    new_cols = map(lambda x: x.replace(' ', '').replace('\\', '') , list(df.columns))
    df.columns = new_cols
    VALID_COLS = ['age,unit,sex,geotime', '2016', '2015', '2014', '2013', '2012']
    df = df[VALID_COLS]
    df = df.replace(': ', np.nan)
    str_to_cols = pd.DataFrame(df['age,unit,sex,geotime']
                               .str.split(',').tolist(), 
                               columns='age,unit,sex,geotime'.split(','))
    
    df = pd.concat([df, str_to_cols], axis=1)
    df = df[df['geotime'].apply(lambda x: x in VALID_EU.values())]
    df = df[df['unit']=='PC_ACT']
    
    return df
        
raw_unemployment_eu = pd.read_csv(UNEMPLOYMENT_EU_PATH, sep='\t')
unemployment_eu = treat_raw_unemployment(raw_unemployment_eu)

In [3]:
topojson_eu = json.load(open(TOPOJSON_EU_PATH))
topojson_ch = json.load(open(TOPOJSON_CH_PATH))

In [4]:
def treat_and_filter_eu(topojson):
    '''filter EU and fix some ids that are not equal the standard in the topojson'''
    new_topojson = copy.deepcopy(topojson)
    eu_countries = VALID_EU.values()
    geometries =  new_topojson['objects']['europe']['geometries']
    
    for g in geometries:
        if g['id']=='GB':
            g['id'] = 'UK'
        if g['id']=='GR':
            g['id'] = 'EL'
            
    filtered_eu = list(filter(lambda x: x['id'] in eu_countries, geometries))
    new_topojson['objects']['europe']['geometries'] = filtered_eu
    return new_topojson
                      
filtered_topojson_eu = treat_and_filter_eu(topojson_eu)

In [349]:
map_eu = folium.Map(location=MIDDLE_EU, tiles='Mapbox Bright', zoom_start=4)

map_eu.choropleth(geo_data=filtered_topojson_eu, data=unemployment_eu, 
                  columns=['geotime', '2016'],
                  key_on='feature.id',
                  topojson='objects.europe',
                  fill_color='YlOrRd', 
                  fill_opacity=0.7, 
                  line_opacity=0.8,
                  legend_name='Unemployment 2016 EU',)
map_eu.save(os.path.join(FIGURES_FOLDER, 'unemployment_eu.html'))

### EU unemployment
Check the html file [here](figures/unemployment_eu.html)

As expected Greece has the highest unemployment rate which is consistency with the news from the last year.
Italy and Spain also have high rates and are having problems since crisis euro to revert this scenario.

In [12]:
CH_UNEMPLOYMENT_2016 = 3.32

unemployment_eu['compare_ch_2016'] = (unemployment_eu['2016'] - CH_UNEMPLOYMENT_2016) / CH_UNEMPLOYMENT_2016

map_eu_ch = folium.Map(location=MIDDLE_EU, tiles='Mapbox Bright', zoom_start=4)
map_eu_ch.choropleth(geo_data=filtered_topojson_eu, data=unemployment_eu, 
                     columns=['geotime', 'compare_ch_2016'],
                     key_on='feature.id',
                     topojson='objects.europe',
                     fill_color='YlOrRd', 
                     fill_opacity=0.7, 
                     line_opacity=0.8,
                     legend_name='Unemployment 2016 EU relative by Switzerland Unemployment',)
map_eu_ch.save(os.path.join(FIGURES_FOLDER, 'unemployment_eu_compare_ch.html'))

### EU unemployment compared with Switzerland
Check the html file [here](figures/unemployment_eu_compare_ch.html)

It is hard to compare other countries from EU with Switzerland because the definition of unemployment is slighly different and it is mesured in a different way too. Besides that, as Switzerland is a small country, it can have a better accuracy.

Considering that the differences cited above are irrelevant, all countries from EU has higher an unemployment rate higher than Switzerland and the chart just rescale the raw rates. 

## 2

Go to the [amstat](https://www.amstat.ch) website to find a dataset that includes the unemployment rates in Switzerland at a recent date.

 Use this data to build another Choropleth map, this time showing the unemployment rate at the level of swiss cantons. Again, try to make the map as expressive as possible, and comment on the trends you observe.

   The Swiss Confederation defines the rates you have just plotted as the number of people looking for a job divided by the size of the active population (scaled by 100). This is surely a valid choice, but as we discussed one could argue for a different categorization.

   Copy the map you have just created, but this time don't count in your statistics people who already have a job and are looking for a new one. How do your observations change ? You can repeat this with different choices of categories to see how selecting different metrics can lead to different interpretations of the same data.

